In [16]:
import os
import time
import math
import pandas as pd
import numpy as np
from numbers import Number
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import visdom
from torch.autograd import Variable
from torch.utils.data import DataLoader
# import plotly.express as px
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import shutil

import lib.dist as dist
import lib.utils as utils
# import lib.datasets as dset
from lib.flows import FactorialNormalizingFlow

from elbo_decomposition import elbo_decomposition
from plot_latent_vs_true import plot_vs_gt_shapes, plot_vs_gt_faces

ModuleNotFoundError: No module named 'pandas'

In [1]:
for filename in os.listdir('/home/desai.ven/TCVAE/img_align_celeba'):
            # image = Image.open(os.path.join("images", filename))
            image = Image.open(os.path.join('/home/desai.ven/TCVAE/img_align_celeba', filename))
            image = transform(image)
            self.dataset.append(image)

'/home/desai.ven/TCVAE'

In [2]:
root='/home/desai.ven/TCVAE/img_align_celeba'
img_list=os.listdir(root)
print(len(img_list))

202599


In [1]:
import pandas as pd

In [2]:
att=pd.read_csv('/home/desai.ven/TCVAE/list_attr_celeba.csv')

In [7]:
att.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [16]:
att=att[["image_id","Bald","Oval_Face","Male","Mustache"]]

In [18]:
for i, (_,row) in enumerate(att.iterrows()):
    if row["Bald"]==1:
        shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Bald/"+row["image_id"] )
    if row["Oval_Face"]==1:
        shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Oval_Face/"+row["image_id"] )
    if row["Male"]==1:
        shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Male/"+row["image_id"] )
    if row["Mustache"]==1:
        shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Mustache/"+row["image_id"] )

In [19]:
img_list=os.listdir("/home/desai.ven/TCVAE/celeba_att/Bald/")
img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Oval_Face/"))
img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Male/"))
img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Mustache/"))

In [21]:
# print(len(img_list))

154965


In [15]:
image_size = 64
pix_mean = [0.5, 0.5, 0.5]
pix_std = [0.5, 0.5, 0.5]

celeba_dataset = datasets.ImageFolder(
    root='/home/desai.ven/TCVAE/celeba_att',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(pix_mean, pix_std),
    ]))

In [ ]:
print(len(celeba_dataset))

In [ ]:
batch_size=2048
num_worksers=4
pin_memory= use_cuda

In [ ]:
train_loader=torch.utils.data.DataLoader(
    celeba_dataset,
    batch_size=2048,
    shuffle=True,
    num_workers=num_worksers,
    pin_memory=pin_memory
)

In [ ]:
prior_dist = dist.Normal()
q_dist = dist.Normal()

In [ ]:
num_epochs=50
lr=1e-3
latent_dim=10
tcvae=True
conv= True
mss= False
beta=15
lamb=0
log_freq=200
save="results_celeba_Bald"

In [ ]:
vae = VAE(z_dim=latent_dim, use_cuda=True, prior_dist=prior_dist, q_dist=q_dist, include_mutinfo=True, tcvae=tcvae, conv=conv, mss=mss)

In [ ]:
optimizer = optim.Adam(vae.parameters(), lr=lr)
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * num_epochs
iteration = 0
    # initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
        for i, x in enumerate(train_loader):
            iteration += 1
            batch_time = time.time()
            vae.train()
            optimizer.zero_grad()
            x = x.cuda()
            # wrap the mini-batch in a PyTorch Variable
            x = Variable(x)
            # do ELBO gradient and accumulate loss
            obj, elbo = vae.elbo(x, dataset_size)
            if utils.isnan(obj).any():
                raise ValueError('NaN spotted in objective.')
            obj.mean().mul(-1).backward()
            # elbo_running_mean.update(elbo.mean().data[0])
            elbo_running_mean.update(elbo.mean().data.item())
            optimizer.step()
            if iteration % args.log_freq == 0:
                train_elbo.append(elbo_running_mean.avg)
                print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                    iteration, time.time() - batch_time, vae.beta, vae.lamb,
                    elbo_running_mean.val, elbo_running_mean.avg))

                vae.eval()
                utils.save_checkpoint({'state_dict': vae.state_dict()}, save, 0)

vae.eval()
utils.save_checkpoint({'state_dict': vae.state_dict()}, save, 0)
            


Defaulting to user installation because normal site-packages is not writeable
